In [1]:
import pandas as pd
import numpy as py
df = pd.read_csv("finaldataset.csv")

df['Tag1'].unique()

array(["['Civil claims']", "['Incapacity and inheritance']",
       "['Personal legal procedures']", "['Property and Housing']",
       "['Family']", nan,
       "['Contentious Commercial Corporate Disputes ']", "['Crime']",
       "['TMT Data Protection and Privacy']",
       "['Non Contentious Commercial Corporate Advisory ']",
       "['Intellectual Property']", "['Syariah Law']"], dtype=object)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn import naive_bayes

from sklearn.multiclass import OneVsRestClassifier

In [4]:
multilabel = MultiLabelBinarizer()
y = multilabel.fit_transform(ex['Tag3'])
y

NameError: name 'ex' is not defined

In [ ]:
df['Case'].replace('', py.nan, inplace=True)
df.dropna(subset=['Case'], inplace=True)
df['Case'].isnull().sum()

df['Tag1'].isnull().sum()
df.dropna(subset=['Tag1'], inplace=True)
df['Tag1'].isnull().sum()
df = df[['Case','Tag1']]
df['Tag1'] = df['Tag1'].astype(str)
df['Tag1'].unique()
import ast
df['Tag1'] = df['Tag1'].apply(lambda x: ast.literal_eval(x))
df

In [ ]:
#####################
## Cleaning Text ####
#####################

import re
def casecleaner(casetext):
    casetext = re.sub("[?,.®'&$’\"\-()]"  , "", str(casetext)) #remove special characters
    casetext = re.sub("�<[a-z]+/?>", "", str(casetext)) #remove html codes
    casetext = re.sub("[^\x00-\x7F]+", "", str(casetext))
    casetext = casetext.strip() #remove leading and trailing spaces
    casetext = casetext.lower() #change to lower case
    return casetext

df['Case'] = df['Case'].apply(casecleaner)
df.head()

##################################################
#### Tokenization, Stemming, Stopword Removal ####
##################################################

from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

tokenizer=RegexpTokenizer(r'\w+')
en_stopwords=set(stopwords.words('english'))
ps=PorterStemmer()

def getStemmedReview(review):
    tokens=tokenizer.tokenize(review)
    new_tokens=[token for token in tokens if token not in  en_stopwords]
    stemmed_tokens=[ps.stem(token) for token in new_tokens]
    clean_review=' '.join(stemmed_tokens)
    return clean_review

df

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn import naive_bayes

from sklearn.multiclass import OneVsRestClassifier

In [ ]:
multilabel = MultiLabelBinarizer()
y = multilabel.fit_transform(df['Tag1'])
y

In [ ]:
multilabel.classes_

In [ ]:
pd.DataFrame(y, columns = multilabel.classes_)

In [ ]:
tfidf = TfidfVectorizer(analyzer='word', max_features=1000, ngram_range=(1,3), stop_words='english')
X = tfidf.fit_transform(df['Case'])
X
tfidf.vocabulary_

In [ ]:
X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
def j_score(y_true, y_pred):
    jaccard = py.minimum(y_true, y_pred).sum(axis = 1)/py.maximum(y_true, y_pred).sum(axis = 1)
    return jaccard.mean()*100

def print_score(y_pred, clf):
    print('clf:', clf.__class__.__name__)
    print('Jaccard score: {}'.format(j_score(y_test, y_pred)))

In [ ]:
sgd = SGDClassifier()
lr = LogisticRegression(solver = 'lbfgs')
svc = LinearSVC()
nbc = naive_bayes.MultinomialNB()

clf = OneVsRestClassifier(sgd)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print_score(y_pred, clf)

In [ ]:
for classifier in [sgd, lr, svc, nbc]:
    clf2 = OneVsRestClassifier(classifier)
    clf2.fit(X_train, y_train)
    y_pred = clf2.predict(X_test)
    print_score(y_pred, classifier)

In [ ]:
test_input = ['I have this tenant that rents on my property, a single family house with a back house, she has been renting for 15 years, and we have barely increased her rent for the past 3 years by $50 per year (Yes i know i should have increased it since the beginning), now we are in need of using that section of the home for personal use, but now with the eviction moratorium in California, am i able to evict her out my property with no fault cause? This is also with a non-contract agreement, this was done verbally.']
xt = tfidf.transform(test_input)
clf.predict(xt)
ab = multilabel.inverse_transform(clf.predict(xt))
finalstring = ab[0][0]
finalstring
print(clf.predict(xt))

In [ ]:
import joblib

joblib.dump(clf, 'model.pkl')
joblib.dump(tfidf, 'tfidf.pkl')